In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from load import load_pseudo, load_nucleotides

pd.options.display.precision = 3
pd.options.display.max_colwidth = 10

%matplotlib inline

In [2]:
%time records = load_pseudo()
numerical_response = pd.read_csv('../data/pseudo/Perron_phenotype-GSU-training.csv')
records = records.merge(numerical_response[['strain', 'carb.lag.delta', 'toby.lag.delta']],
                        left_on='lab-id', right_on='strain', how='left')
records.rename(columns={'carb.lag.delta': 'carb_num', 'toby.lag.delta': 'toby_num'}, inplace=True)
records.drop(columns=['strain', 'lab-id'], inplace=True)
records.head()

CPU times: user 4.43 s, sys: 106 ms, total: 4.54 s
Wall time: 4.56 s


,id,sequence,missing,missing_%,sequence_i,missing_i,missing_%_i,carb,toby,carb_num,toby_num
0,TA151,ATGAGT...,31842,6.588,ATGAGT...,28410,5.878,True,False,-2.0,16.0
1,IC1,ATGAGT...,46071,9.532,ATGAGT...,34714,7.182,False,False,2.0,14.0
2,A237,ATGAGT...,44514,9.210,ATGAGT...,35933,7.434,True,False,-1.0,4.0
3,5920,ATGAGT...,49497,10.241,ATGAGT...,36873,7.629,NaN,NaN,NaN,NaN
4,LiA96,ATGAGT...,44067,9.117,ATGAGT...,34454,7.128,False,False,0.0,18.0


In [3]:
mask = (records['toby'].notna() & records['carb'].notna())

# Feature selection

In [4]:
o_n = np.load('../data/pseudo/preprocess/o_n_-_-.npy')
i_n = np.load('../data/pseudo/preprocess/i_n_-_-.npy')

In [5]:
# 40 seconds
%time o_n = load_nucleotides('../data/pseudo/concatenated.fasta')
%time i_n = load_nucleotides('../data/pseudo/concatenated_naive_impute.fasta')

In [6]:
# 45 seconds
forward = str.maketrans('-ACTG', '01234')
def transformation(str):
    return [int(i) for i in str.translate(forward)]
%time o_n = pd.DataFrame(records['sequence'].apply(transformation).to_list())
%time i_n = pd.DataFrame(records['sequence_i'].apply(transformation).to_list())
np.save('../data/pseudo/preprocess/o_n_-_-.npy', o_n)
np.save('../data/pseudo/preprocess/i_n_-_-.npy', i_n)

## Variance threshold

In [7]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(0.01)

# justification(not rigorous) for why < 0.016 is the threshold to drop a column
a, b = 4, 3
arr = np.ones((122, 1))*a
arr[:2] = b
np.var(arr)

o_n_v = selector.fit_transform(o_n)
o_n_v_selected = pd.Series(selector.get_support())
o_n_v_selected.value_counts()

i_n_v = selector.fit_transform(i_n)
i_n_v_selected = pd.Series(selector.get_support())
i_n_v_selected.value_counts()

## Remove based on SNP counts
similar to variance threshold but seems better

In [8]:
# less than 6 min
%time snp_counts_o = o_n.apply(pd.Series.value_counts, axis=0)
%time snp_counts_i = i_n.apply(pd.Series.value_counts, axis=0)
np.save('../data/pseudo/preprocess/others/snp_counts_o.npy', snp_counts_o.to_numpy())
np.save('../data/pseudo/preprocess/others/snp_counts_i.npy', snp_counts_i.to_numpy())

snp_counts_o = pd.DataFrame(np.load('../data/pseudo/preprocess/others/snp_counts_o.npy'))
snp_counts_i = pd.DataFrame(np.load('../data/pseudo/preprocess/others/snp_counts_i.npy'))

# True     204101
snp_max_counts_o = snp_counts_o.max()
(snp_max_counts_o<121).value_counts()

# True     204101
snp_max_counts_i = snp_counts_i.max()
(snp_max_counts_o<121).value_counts()

o_n_v = o_n[mask].loc[:, (snp_max_counts_o<121)]
i_n_v = i_n[mask].loc[:, (snp_max_counts_i<121)]
np.save('../data/pseudo/preprocess/o_n_v_-.npy', o_n_v)
np.save('../data/pseudo/preprocess/i_n_v_-.npy', i_n_v)

## $\chi^2$ on the previous result
because some features are all 0's, so gives `divide by 0` warning

no warning if we remove those features (on the previous step)

In [9]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

o_n_x = SelectKBest(chi2, k=250032//2).fit_transform(o_n_v, records['toby'][mask].astype('i4'))
i_n_x = SelectKBest(chi2, k=105124//2).fit_transform(i_n_v, records['toby'][mask].astype('i4'))
np.save('../data/pseudo/preprocess/o_n_x_-.npy', o_n_x)
np.save('../data/pseudo/preprocess/i_n_x_-.npy', i_n_x)

# Feature extraction

In [10]:
o_n_v = np.load('../data/pseudo/preprocess/o_n_v_-.npy')
i_n_v = np.load('../data/pseudo/preprocess/i_n_v_-.npy')
o_n_x = np.load('../data/pseudo/preprocess/o_n_x_-.npy')
i_n_x = np.load('../data/pseudo/preprocess/i_n_x_-.npy')

## String kernel

In [11]:
from strkernel.mismatch_kernel import MismatchKernel

# 9 minutes
%time o_n__s = MismatchKernel(l=5, k=4, m=1).get_kernel(o_n)
%time i_n__s = MismatchKernel(l=5, k=4, m=1).get_kernel(i_n)
np.save('../data/pseudo/preprocess/o_n_-_s.npy', o_n__s.kernel)
np.save('../data/pseudo/preprocess/i_n_-_s.npy', i_n__s.kernel)

# 3 and 1 minutes
%time o_n_v_s = MismatchKernel(l=5, k=4, m=1).get_kernel(o_n_v)
%time i_n_v_s = MismatchKernel(l=5, k=4, m=1).get_kernel(i_n_v)
np.save('../data/pseudo/preprocess/o_n_v_s.npy', o_n_v_s.kernel)
np.save('../data/pseudo/preprocess/i_n_v_s.npy', i_n_v_s.kernel)

# 2 minutes
%time o_n_x_s = MismatchKernel(l=5, k=4, m=1).get_kernel(o_n_x)
%time i_n_x_s = MismatchKernel(l=5, k=4, m=1).get_kernel(i_n_x)
np.save('../data/pseudo/preprocess/o_n_x_s.npy', o_n_x_s.kernel)
np.save('../data/pseudo/preprocess/i_n_x_s.npy', i_n_x_s.kernel)

## PCA

In [12]:
from sklearn.decomposition import PCA

%time o_n__p = PCA(n_components=119).fit_transform(o_n)
%time i_n__p = PCA(n_components=119).fit_transform(i_n)
np.save('../data/pseudo/preprocess/o_n_-_p.npy', o_n__p)
np.save('../data/pseudo/preprocess/i_n_-_p.npy', i_n__p)

%time o_n_v_p = PCA(n_components=119).fit_transform(o_n_v)
%time i_n_v_p = PCA(n_components=119).fit_transform(i_n_v)
np.save('../data/pseudo/preprocess/o_n_v_p.npy', o_n_v_p)
np.save('../data/pseudo/preprocess/i_n_v_p.npy', i_n_v_p)

%time o_n_x_p = PCA(n_components=119).fit_transform(o_n_x)
%time i_n_x_p = PCA(n_components=119).fit_transform(i_n_x)
np.save('../data/pseudo/preprocess/o_n_x_p.npy', o_n_x_p)
np.save('../data/pseudo/preprocess/i_n_x_p.npy', i_n_x_p)

## T-SNE

In [13]:
from sklearn.manifold import TSNE

%time o_n__t = TSNE(n_components=3).fit_transform(o_n)
%time i_n__t = TSNE(n_components=3).fit_transform(i_n)
np.save('../data/pseudo/preprocess/o_n_-_t.npy', o_n__t)
np.save('../data/pseudo/preprocess/i_n_-_t.npy', i_n__t)

%time o_n_v_t = TSNE(n_components=3).fit_transform(o_n_v)
%time i_n_v_t = TSNE(n_components=3).fit_transform(i_n_v)
np.save('../data/pseudo/preprocess/o_n_v_t.npy', o_n_v_t)
np.save('../data/pseudo/preprocess/i_n_v_t.npy', i_n_v_t)

%time o_n_x_t = TSNE(n_components=3).fit_transform(o_n_x)
%time i_n_x_t = TSNE(n_components=3).fit_transform(i_n_x)
np.save('../data/pseudo/preprocess/o_n_x_t.npy', o_n_x_t)
np.save('../data/pseudo/preprocess/i_n_x_t.npy', i_n_x_t)

# Machine learning

In [14]:
import os
import pickle
from sklearn.preprocessing import OneHotEncoder

random_state = 42

In [71]:
# read the numerical data and use onehot encoder
s = {'{}_n_{}_{}.npy'.format(impute, selection, extraction)
     for impute in 'io'
     for selection in '-vx'
     for extraction in '-pts'}

data = {d: np.load(os.path.join('../data/pseudo/preprocess', d)) for d in s}
# mask all data to remove x with NAN labels
for k, v in data.items():
    if v.shape[0] != 119:
        data[k] = v[mask]

for file, X in data.items():
    encoder = OneHotEncoder(categories='auto', sparse=False, dtype=np.int32)
    %time X_encode = encoder.fit_transform(X)
    
    np.save(os.path.join('../data/pseudo/preprocess/onehot', file), X_encode)
    with open(os.path.join('../data/pseudo/preprocess/onehot-encoder', file[:file.index('.')]), 'wb') as output:
        pickle.dump(encoder, output)

In [79]:
data_encode = {d: np.load(os.path.join('../data/pseudo/preprocess/onehot', d)) for d in s}

In [82]:
X = data_encode['i_n_v_p.npy']
y = records['carb'][mask].astype('?')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state, stratify=y, train_size=0.6)

## Regression

In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [111]:
param_grid = {'C': [0.1, 1, 10], 'class_weight': [None, 'balanced'], 'l1_ratio': [0.,0.5, 1.]}
clf = GridSearchCV(LogisticRegression(penalty='elasticnet', solver='saga', max_iter=2000, verbose=True, n_jobs=3),
                   param_grid=param_grid, scoring='accuracy',
                   cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state))

In [ ]:
clf.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 1292 epochs took 128 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 1151 epochs took 114 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

/mnt/ws/home/zzhang3/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.0min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 308 seconds


/mnt/ws/home/zzhang3/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 309 seconds


/mnt/ws/home/zzhang3/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
clf.cv_results_

In [100]:
clf.best_estimator_

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)